In [ ]:
#import tensorflow as tf
#import os

In [ ]:
#if 'COLAB_TPU_ADDR' in os.environ:
#  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
#else:
#  TF_MASTER =''

#tpu_address = TF_MASTER
#print(tpu_address)

In [ ]:
#resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
#tf.config.experimental_connect_to_cluster(resolver)
#tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.optim as optim
import os

device_gpu = torch.device("cuda:0") if torch.cuda.is_available else ("cpu") 
print(device_gpu)

cuda:0


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
if (not os.path.isfile('eng_consolidated_file.txt')):
  #Download a file based on its id
  file_id = '1mtAaJ1TgoRiNtWJp5IiuOyUxTMF1xjkg' #https://colab.research.google.com/drive/1RzzAb7fV60h8SKT6bsRYeGtPPwqbXXPi?usp=sharing
  downloaded = drive.CreateFile({'id':file_id})
  #print('Downloaded content "{}"'.format(downloaded.GetContentString()))
  #downloaded.GetContentFile('eng_consolidated_file.txt')
  downloaded.GetContentFile(filename='eng_consolidated_file.txt')

if (not os.path.isfile('hindi_consolidated_file.txt')):
  #Download a file based on its id
  file_id = '1ItVlaPLaPBmQTyPyMGFaT-SZAx4Fjcqw' #https://colab.research.google.com/drive/1RzzAb7fV60h8SKT6bsRYeGtPPwqbXXPi?usp=sharing
  downloaded = drive.CreateFile({'id':file_id})
  #print('Downloaded content "{}"'.format(downloaded.GetContentString()))
  #downloaded.GetContentFile('eng_consolidated_file.txt')
  downloaded.GetContentFile(filename='hindi_consolidated_file.txt')


###Prepare dataframe

In [ ]:
with open(os.path.join("/content/", "hindi_consolidated_file.txt"), "r", encoding="utf-8") as f:
  hindi_text = f.readlines()
  hindi_text = [text.strip("\n") for text in hindi_text]

with open(os.path.join("/content/", "eng_consolidated_file.txt"), "r") as f:
  english_text = f.readlines()
  english_text = [text.strip("\n") for text in english_text]

data = []
i = 0
for hindi, english in zip(hindi_text, english_text):
  data.append(["translate english to hindi",english, hindi])
  data.append(["translate hindi to english",hindi, english])
  i = i+1

full_df = pd.DataFrame(data, columns=["prefix","input_text", "target_text"])


In [ ]:
full_df.iloc[0:4]

,prefix,input_text,target_text
0,translate english to hindi,"Questioner: Sadhguru, Namaskaram (Pause).","प्रश्नकर्ता: सद्गुरु, नमस्कारम्।"
1,translate hindi to english,"प्रश्नकर्ता: सद्गुरु, नमस्कारम्।","Questioner: Sadhguru, Namaskaram (Pause)."
2,translate english to hindi,"Sadhguru, how to conduct oneself when, you kno...","सद्गुरु, अगर हमारा... जीवनसाथी बहुत शोषण करने ..."
3,translate hindi to english,"सद्गुरु, अगर हमारा... जीवनसाथी बहुत शोषण करने ...","Sadhguru, how to conduct oneself when, you kno..."


###Take 20% of data for evaluation and remaining for fine-tuning

In [ ]:
from sklearn.model_selection import train_test_split

#For sacrebleu
training_data, testing_data = train_test_split(full_df, test_size=0.2, random_state=25)

#For fine-tuning
train_data, eval_data = train_test_split(training_data, test_size=0.2, random_state=25)
print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")
training_data.iloc[2:3]
testing_data.iloc[2:3]

No. of training examples: 11934
No. of testing examples: 2984


,prefix,input_text,target_text
12419,translate hindi to english,स्प्रिंग का मतलब ये कभी न खत्म होने वाली प्रक्...,"A spiral means it’s a never ending process, ju..."


###Download and unzip Output folder
###

In [ ]:
if (not os.path.isfile('outputs.zip')):
  #Download a file based on its id
  file_id = '1oUMUsx4neB-D1SS9YVlcTAp-5YN7gRWK' #https://colab.research.google.com/drive/1RzzAb7fV60h8SKT6bsRYeGtPPwqbXXPi?usp=sharing
  downloaded = drive.CreateFile({'id':file_id})
  #print('Downloaded content "{}"'.format(downloaded.GetContentString()))
  downloaded.GetContentFile('outputs.zip')

if (not os.path.isdir('outputs')):
  !unzip outputs.zip -d outputs

Archive:  outputs.zip
   creating: outputs/best_model/
  inflating: outputs/best_model/config.json  
 extracting: outputs/best_model/eval_results.txt  
  inflating: outputs/best_model/model_args.json  
  inflating: outputs/best_model/optimizer.pt  
  inflating: outputs/best_model/pytorch_model.bin  
  inflating: outputs/best_model/scheduler.pt  
  inflating: outputs/best_model/special_tokens_map.json  
  inflating: outputs/best_model/spiece.model  
  inflating: outputs/best_model/tokenizer_config.json  
  inflating: outputs/best_model/training_args.bin  
   creating: outputs/checkpoint-14400-epoch-1/
  inflating: outputs/checkpoint-14400-epoch-1/config.json  
  inflating: outputs/checkpoint-14400-epoch-1/model_args.json  
  inflating: outputs/checkpoint-14400-epoch-1/optimizer.pt  
  inflating: outputs/checkpoint-14400-epoch-1/pytorch_model.bin  
  inflating: outputs/checkpoint-14400-epoch-1/scheduler.pt  
  inflating: outputs/checkpoint-14400-epoch-1/special_tokens_map.json  
  inflat

###Create Mt5 model and initialize wandB

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 247 kB 7.2 MB/s 
     |████████████████████████████████| 306 kB 50.5 MB/s 
     |████████████████████████████████| 9.3 MB 45.2 MB/s 
     |████████████████████████████████| 3.4 MB 54.7 MB/s 
     |████████████████████████████████| 6.8 MB 52.7 MB/s 
     |████████████████████████████████| 1.2 MB 48.9 MB/s 
     |████████████████████████████████| 1.7 MB 52.1 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 3.3 MB 40.6 MB/s 
     |████████████████████████████████| 596 kB 39.0 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 57.3 MB/s 
     |████████████████████████████████| 142 kB 65.1 MB/s 
     |████████████████████████████████| 180 kB 62.5 MB/s 
     |████████████████████████████████| 97 kB 6.8 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 1.1 MB 61.2 MB/s 
     |█████████████████

In [ ]:
training_data.head(5)

,prefix,input_text,target_text
2884,translate english to hindi,"This is not a developed country, “What shall I...","यह विकसित देश नहीं है, ‘मैं क्या करूं, सब कुछ ..."
5305,translate hindi to english,"पहले उनका हरण कर लेते हैं, नहीं?","First let’s kidnap her, no (Laughter/Applause)?"
13301,translate hindi to english,सद्गुरु: मान लीजिए कि आपका दायाँ हाथ अजीब हरकत...,Sadhguru: Suppose your right hand gets up and ...
10201,translate hindi to english,"वरना गिरना बहुत बढ़िया है, जब तक आप जमीन से न ट...","Otherwise, falling is very nice, till you hit ..."
129,translate hindi to english,"या तो आपके पास इसे बदलने का साहस होना चाहिए, य...",Either you must have the courage to change it ...


In [ ]:
testing_data = testing_data.reset_index()
training_data = training_data.reset_index()

In [ ]:
print(testing_data.shape)
print(training_data.shape)

(2984, 4)
(11934, 4)


In [ ]:
training_data.head(5)

,index,prefix,input_text,target_text
0,2884,translate english to hindi,"This is not a developed country, “What shall I...","यह विकसित देश नहीं है, ‘मैं क्या करूं, सब कुछ ..."
1,5305,translate hindi to english,"पहले उनका हरण कर लेते हैं, नहीं?","First let’s kidnap her, no (Laughter/Applause)?"
2,13301,translate hindi to english,सद्गुरु: मान लीजिए कि आपका दायाँ हाथ अजीब हरकत...,Sadhguru: Suppose your right hand gets up and ...
3,10201,translate hindi to english,"वरना गिरना बहुत बढ़िया है, जब तक आप जमीन से न ट...","Otherwise, falling is very nice, till you hit ..."
4,129,translate hindi to english,"या तो आपके पास इसे बदलने का साहस होना चाहिए, य...",Either you must have the courage to change it ...


In [ ]:
training_data = training_data.drop("index", axis=1)
testing_data = testing_data.drop("index", axis=1)

In [ ]:
testing_data.head(5)

,prefix,input_text,target_text
0,translate hindi to english,क्या ये इतनी बड़ी नहीं कि इसे नमन किया जाए?,Isn't it big enough to bow down?
1,translate hindi to english,"Popular, hmm? एक बार की बात है, शंकरन पिल्लै...","Popular, hmm (Laughter)?It once happened, Shan..."
2,translate hindi to english,स्प्रिंग का मतलब ये कभी न खत्म होने वाली प्रक्...,"A spiral means it’s a never ending process, ju..."
3,translate hindi to english,समाज में शादी होनी चाहिए या नहीं - इस प्रश्न प...,Whether there should be marriage in society or...
4,translate english to hindi,Suddenly you find you’re a stranger in your ow...,अचानक अभ्यास के दो दिनों बाद ही आप अपने ही घर ...


In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 7.9 MB/s 


In [ ]:
from simpletransformers.t5 import T5Model, T5Args
model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 20
model_args.eval_batch_size = 20
model_args.num_train_epochs = 2
model_args.evaluate_during_training = True
#model_args.evaluate_during_training_steps = 30000 #change from original code
model_args.evaluate_during_training_steps = 60
#model_args.evaluate_during_training_steps = 10000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Hindi-English Translation_v1_24Sept"

model = T5Model("mt5", "google/mt5-small", args=model_args)

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

In [ ]:
print(training_data.shape)
print(testing_data.shape)

(11934, 3)
(2984, 3)


In [ ]:
training_data.head(5)

,prefix,input_text,target_text
0,translate english to hindi,"This is not a developed country, “What shall I...","यह विकसित देश नहीं है, ‘मैं क्या करूं, सब कुछ ..."
1,translate hindi to english,"पहले उनका हरण कर लेते हैं, नहीं?","First let’s kidnap her, no (Laughter/Applause)?"
2,translate hindi to english,सद्गुरु: मान लीजिए कि आपका दायाँ हाथ अजीब हरकत...,Sadhguru: Suppose your right hand gets up and ...
3,translate hindi to english,"वरना गिरना बहुत बढ़िया है, जब तक आप जमीन से न ट...","Otherwise, falling is very nice, till you hit ..."
4,translate hindi to english,"या तो आपके पास इसे बदलने का साहस होना चाहिए, य...",Either you must have the courage to change it ...


In [ ]:
testing_data.head(5)

,prefix,input_text,target_text
0,translate hindi to english,क्या ये इतनी बड़ी नहीं कि इसे नमन किया जाए?,Isn't it big enough to bow down?
1,translate hindi to english,"Popular, hmm? एक बार की बात है, शंकरन पिल्लै...","Popular, hmm (Laughter)?It once happened, Shan..."
2,translate hindi to english,स्प्रिंग का मतलब ये कभी न खत्म होने वाली प्रक्...,"A spiral means it’s a never ending process, ju..."
3,translate hindi to english,समाज में शादी होनी चाहिए या नहीं - इस प्रश्न प...,Whether there should be marriage in society or...
4,translate english to hindi,Suddenly you find you’re a stranger in your ow...,अचानक अभ्यास के दो दिनों बाद ही आप अपने ही घर ...


In [ ]:
model = T5Model("mt5", "/content/outputs", args=model_args)
model.train_model(training_data, eval_data=testing_data)

  0%|          | 0/11934 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3426: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


...

  0%|          | 0/2984 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3426: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3426: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the 

  0%|          | 0/2984 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3426: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3426: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the 

(1194,
 {'eval_loss': [1.975185407002767,
   1.8699664751688638,
   1.797164684931437,
   1.7624518736203512,
   1.7299328629175823,
   1.703730407555898,
   1.67746688524882,
   1.649225561618805,
   1.631660528977712,
   1.6237750021616617,
   1.6233251587549846,
   1.613507784207662,
   1.5976259668668111,
   1.5846685397624969,
   1.5805279874801637,
   1.570696803331375,
   1.5660810844103494,
   1.55214328845342,
   1.5409642159938812,
   1.5285427029927572,
   1.524176001548767],
  'global_step': [60,
   120,
   180,
   240,
   300,
   360,
   420,
   480,
   540,
   597,
   600,
   660,
   720,
   780,
   840,
   900,
   960,
   1020,
   1080,
   1140,
   1194],
  'train_loss': [2.1275696754455566,
   1.6122742891311646,
   2.2731101512908936,
   1.8360309600830078,
   1.7781572341918945,
   1.5377223491668701,
   1.8855890035629272,
   1.9080510139465332,
   1.4902853965759277,
   1.5103046894073486,
   1.549634337425232,
   1.8857250213623047,
   1.701446294784546,
   1.62620

#Evaluate using sacrebleu

In [ ]:
!pip install sacrebleu 

     |████████████████████████████████| 90 kB 5.2 MB/s 


In [ ]:
import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model_args = T5Args()
model_args.max_length = 512
model_args.length_penalty = 1
model_args.num_beams = 10

model = T5Model("mt5", "outputs", args=model_args)

In [ ]:
hindi_truth = testing_data["target_text"].tolist()
to_hindi =  testing_data["input_text"].tolist()

Get input_text from eval_df 
Get hindi_truth
Pass it to hindi_preds = model.predict(to_english)
#

In [ ]:
# Predict
hindi_preds = model.predict(to_hindi)

eng_sin_bleu = sacrebleu.corpus_bleu(hindi_preds, hindi_truth)
print("--------------------------")
print("English to Sinhalese: ", eng_sin_bleu.score)

Generating outputs:   0%|          | 0/373 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3426: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/2984 [00:00<?, ?it/s]

--------------------------
English to Sinhalese:  0.7885032935726018


In [ ]:
print(model.predict(["first of all, he left Vrindavan, never again came back."]))

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3426: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['सबसे पहले उन्होंने वर्नरवन को छोड़ दिया, फिर कभी वापस नहीं आया।']


#Do translation for actual english file

In [ ]:
#with open(os.path.join(data_path, "train.trg"), "r", encoding="utf-8") as f:
#  hindi_text = f.readlines()

#Read the english file in text
with open(os.path.join("/content/" , "eng_lineByLine.txt"), "r") as f:
  english_text=f.readlines()

#For each line in english_text, call the function to translate. Save the output of the function to another list
hindi_text = []
#for english_line in english_text:
#  hindi_line = model.predict(english_line)
#  hindi_text.append(hindi_line)
hindi_text = model.predict(english_text)   

Generating outputs:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3399: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
textfile = open("hindi_lineByLine.txt", "wb")
sample_str = " "
for hindi_line in hindi_text:
    hindi_line =  hindi_line + "\n"
    encoded_unicode = hindi_line.encode("utf8")
    print(hindi_line)
    print(encoded_unicode)
    textfile.write(encoded_unicode)
textfile.close()    

इंसान सद्गुरु, जीवन का मकसद क्या है? सद्गुरु - कोई मकसद नहीं है।

b'\xe0\xa4\x87\xe0\xa4\x82\xe0\xa4\xb8\xe0\xa4\xbe\xe0\xa4\xa8 \xe0\xa4\xb8\xe0\xa4\xa6\xe0\xa5\x8d\xe0\xa4\x97\xe0\xa5\x81\xe0\xa4\xb0\xe0\xa5\x81, \xe0\xa4\x9c\xe0\xa5\x80\xe0\xa4\xb5\xe0\xa4\xa8 \xe0\xa4\x95\xe0\xa4\xbe \xe0\xa4\xae\xe0\xa4\x95\xe0\xa4\xb8\xe0\xa4\xa6 \xe0\xa4\x95\xe0\xa5\x8d\xe0\xa4\xaf\xe0\xa4\xbe \xe0\xa4\xb9\xe0\xa5\x88? \xe0\xa4\xb8\xe0\xa4\xa6\xe0\xa5\x8d\xe0\xa4\x97\xe0\xa5\x81\xe0\xa4\xb0\xe0\xa5\x81 - \xe0\xa4\x95\xe0\xa5\x8b\xe0\xa4\x88 \xe0\xa4\xae\xe0\xa4\x95\xe0\xa4\xb8\xe0\xa4\xa6 \xe0\xa4\xa8\xe0\xa4\xb9\xe0\xa5\x80\xe0\xa4\x82 \xe0\xa4\xb9\xe0\xa5\x88\xe0\xa5\xa4\n'
एक पूरा-पूरा जीवन बनना, अपने पूरे जीवन में जीवन को जानना, अपने पूरे जीवन में जीना सबसे बड़ा मकसद है।

b'\xe0\xa4\x8f\xe0\xa4\x95 \xe0\xa4\xaa\xe0\xa5\x82\xe0\xa4\xb0\xe0\xa4\xbe-\xe0\xa4\xaa\xe0\xa5\x82\xe0\xa4\xb0\xe0\xa4\xbe \xe0\xa4\x9c\xe0\xa5\x80\xe0\xa4\xb5\xe0\xa4\xa8 \xe0\xa4\xac\xe0\xa4\xa8\xe0\xa4\xa8\xe0\xa4\xbe,

#Try doing translation directly from english text in a list

In [ ]:
hindi_text = model.predict(english_text)

Generating outputs:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3399: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/19 [00:00<?, ?it/s]